# GENERATE PRIVATE KEY

## Generated using the strong RNG(secrets) provided by Python

In [ ]:
import secrets

# create a random 256 address
# take its hex and remoce first two elements (0x)
privateKey1 = hex(secrets.randbits(256))[2:]

print(privateKey1)

6a20747aabd06f0ccafd4cd996f830ad171ab9482de11e433a7e79b6d9f1678a


## Creating a simpler Bitaddress model
  *   Code inspiration from ==> https://www.freecodecamp.org/news/how-to-generate-your-very-own-bitcoin-private-key-7ad0f4936e6c/


Imports




In [ ]:
import time
import random

Generator Class

In [ ]:
class PrivateKeyGenerator:
  def __init__(self):
        self.POOL_SIZE = 256
        self.KEY_BYTES = 32
        self.CURVE_ORDER = int('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141', 16)
        self.pool = [0] * self.POOL_SIZE
        self.pool_pointer = 0
        self.prng_state = None
        self.__init_pool()

# 1st Step : Initializing The Pool
  # Entropy using __seed_int and __seed_byte
  def __init_pool(self):
      for i in range(self.POOL_SIZE):
          # create and seed a random pool of 8 bits
          self.__seed_byte(secrets.randbits(8))

      # time for entropy
      self.__seed_int(int(time.time()))

  def __seed_int(self, n):
      self.__seed_byte(n)
      self.__seed_byte(n >> 8)
      self.__seed_byte(n >> 16)
      self.__seed_byte(n >> 24)

  def __seed_byte(self, n):
      self.pool[self.pool_pointer] ^= n & 255
      self.pool_pointer += 1

      if self.pool_pointer >= self.POOL_SIZE:
          self.pool_pointer = 0

# 2nd Step: Seeding the Input
  # Put a timestamp and then input string
  def seed_input(self, str_input):
      self.__seed_int(int(time.time()))

      for char in str_input:
          self.__seed_byte(ord(char))

# 3rd Step: Generating the Private Key
  # getstate and setstate used to conserve entropy
  # Key of range (1, Curve_Order)
      # Curve_Order is the order of secp256k1
  # Convert to hex and strip ("0x")

  def generate_key(self):
      bigInt = self.__generateBigInt()
      bigInt = bigInt % (self.CURVE_ORDER - 1) # key < curve order
      bigInt = bigInt + 1 # key > 0

      keyHex = hex(bigInt)
      key = keyHex[2:]
      return key

  def __generateBigInt(self):
      if self.prng_state is None:
        seed = int.from_bytes(self.pool, byteorder='big', signed=False)
        random.seed(seed)

      self.prng_state = random.getstate()
      random.setstate(self.prng_state)
      
      bigInt = random.getrandbits(self.KEY_BYTES * 8)
      self.prng_state = random.getstate()
      
      return bigInt

Trial Call

In [ ]:
privKeyGen = PrivateKeyGenerator()
privKeyGen.seed_input('Ranchoddas Shamaldas Chanchad :)')

privateKey2 = privKeyGen.generate_key()
print(privateKey2)

4979e71e76a5350404ca5af7a0b106c5e0cd65e35a7b81f272b34f2928d96275


# GET PUBLIC KEY

Code Inspiration from => https://medium.com/@kootsZhin/step-by-step-guide-to-getting-bitcoin-address-from-private-key-in-python-7ec15072b71b

## Imports

In [ ]:
import hashlib
import base58
import codecs
import ecdsa

## Step 1: Get uncompressed Public Key

In [ ]:
# Hex decoding the private key to bytes using codecs library and Generating a public key in bytes using SECP256k1 & ecdsa library
publicKeyBytes = (ecdsa.SigningKey.from_string(codecs.decode(privateKey2, 'hex'), curve=ecdsa.SECP256k1).verifying_key).to_string()

# Hex encoding the public key from bytes
publicKeyHex = codecs.encode(publicKeyBytes, 'hex')

# Bitcoin public key begins with bytes 0x04 so we have to add the bytes at the start
publicKey = (b'04' + publicKeyHex).decode("utf-8")

print(publicKey)

04f41172a93c094f1984cfedf490617ff86526820b044534c2221fc6de21fc4e4c25a11ef2c95f3a59c6f845a176094dccbd70c9ee5ac824a11fa88dc8994cd952


## Step 2: Get Compressed Public Key

In [ ]:
# Checking if the last byte is odd or even
if (ord(bytearray.fromhex(publicKey[-2:])) % 2 == 0):
    publicKeyCompressed = '02'
else:
    publicKeyCompressed = '03'
    
# Add bytes 0x02 to the X of the key if even or 0x03 if odd
publicKeyCompressed += publicKey[2:66]

print(publicKeyCompressed)

02f41172a93c094f1984cfedf490617ff86526820b044534c2221fc6de21fc4e4c


## Step 3: Perform SHA-256 hashing on the compressed public key

In [ ]:
# Converting to bytearray for SHA-256 hashing
hexStr = bytearray.fromhex(publicKeyCompressed)

# Apply sha
sha = hashlib.sha256()
sha.update(hexStr)
sha.hexdigest() # .hexdigest() is hex ASCII

'433218daec7ea37fd88bd6fe1220254a859ae5436f6f1da320864f457b0f25bf'

## Step 4: Perform RIPMED-160 hashing on the result of SHA-256

In [ ]:
rip = hashlib.new('ripemd160')
rip.update(sha.digest())

keyHash = rip.hexdigest()
print(keyHash) # Hash160

3bbfe5ca08337d6861fbf3104c504dfe80ebc303


## Step 5: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)

In [ ]:
modifiedKeyHash = "00" + keyHash
print(modifiedKeyHash)

003bbfe5ca08337d6861fbf3104c504dfe80ebc303


## Step 6: Perform SHA-256 hash on the extended RIPEMD-160 result (Base58Check encoding)

In [ ]:
# Converting to bytearray for SHA-256 hashing
hexString = bytearray.fromhex(modifiedKeyHash)

# Apply sha
sha2 = hashlib.sha256()
sha2.update(hexString)
sha2.hexdigest()

'b387601c7b426a4dfca774a2918705fb7e0fc36a5a56ea5a4486d71be59c31ca'

## Step 7: Perform SHA-256 hash on the result of the previous SHA-256 hash

In [ ]:
sha3 = hashlib.sha256()
sha3.update(sha.digest())
sha3.hexdigest()

'ea1919789678de9e46a339abbb532e1dc10b00d6a24d417fed58f928c187a5f8'

## Step 8: Take the first 4 bytes of the second SHA-256 hash, this is the address checksum

In [ ]:
checkSum = sha3.hexdigest()[:8]

print(checkSum)

ea191978


## Step 9: Add the 4 checksum bytes from stage 8 at the end of extended RIPEMD-160 hash from stage 5, this is the 25-byte binary Bitcoin Address

In [ ]:
byte25Address = modifiedKeyHash + checkSum

print(byte25Address)

003bbfe5ca08337d6861fbf3104c504dfe80ebc303ea191978


## Step 10 (Final Result): Convert the result from a byte string into a base58 string using Base58Check encoding, this is the most commonly used Bitcoin Address format

In [ ]:
address1 = base58.b58encode(bytes(bytearray.fromhex(byte25Address))).decode('utf-8')

print(address1)

16SvkbgZ4cqeWhTivTExWT4rvoT8Xs9bpj


## ALTERNATIVE WAY: Making use of b58.encode_check from base58 library directly at Step 6

In [ ]:
keyBytes = codecs.decode(modifiedKeyHash, 'hex')
address2 = base58.b58encode_check(keyBytes).decode('utf-8')
address2

'16SvkbgZ4cqeWhTivTExWT4rvoT8Xs9bpj'